In [ ]:
import fooddatacentral as fdc
import warnings
import pandas as pd
import numpy as np
from  scipy.optimize import linprog as lp
import numpy as np
import warnings
import pickle

bulk_prices_w_lbs = pd.read_csv("data/bulk_prices.tsv", sep='\t')
apikey = "YOUR_API_KEY_HERE"

DRIs = None

# note -this replaces the mechanism of the lecture notebooks
# that uses gnugpg credentials to read a Google spreadsheet,
# which has been found to be extremely problematic. A python
# pickle file of the resultant dataframe was created instead
# and stored with the repo

with open('data/diet.pickle', 'rb') as fp:
    DRIs = pickle.load(fp)

# Define *minimums*
diet_min = DRIs['diet_minimums'].set_index('Nutrition')
diet_max = DRIs['diet_maximums'].set_index('Nutrition')

D = {}
count = 0
food_names = bulk_prices_w_lbs.Material_Descr.tolist()
for food_name in food_names:
    try:
        fdc_id = bulk_prices_w_lbs.loc[bulk_prices_w_lbs.Material_Descr==food_name,:].FDC_ID
        int_fdcid = int(fdc_id[count])
        count+=1
        D[food_name] = fdc.nutrients(apikey,int_fdcid).Quantity
        print(f'count: {count} {food_name}')
    except AttributeError:
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food_name,int_fdcid))
    except ValueError:
        warnings.warn(f"No code for food {food_name}")
    except Exception as ex:
        warnings.warn(f"something seriously wrong for {food_name}  {ex}")

FoodNutrients = pd.DataFrame(D,dtype=float)

# all items are 1 pound
bulk_prices_w_lbs['FDC Quantity'] = bulk_prices_w_lbs[['Quantity','BaseUoM']].T.apply(lambda x : fdc.units(x['Quantity'],x['BaseUoM']))
bulk_prices_w_lbs['FDC Price'] = bulk_prices_w_lbs['Average_Price']/bulk_prices_w_lbs['FDC Quantity']

bulk_prices_w_lbs.dropna(how='any')

# To use minimum price observed
Prices = bulk_prices_w_lbs.groupby('Material_Descr',sort=False)['FDC Price'].min()



%store bulk_prices_w_lbs
%store Prices
%store diet_min
%store diet_max

